In [53]:
# 导入各种包
import torch
import torch.nn as nn
from torch.nn import functional as F
import ttach as tta
from resnest.torch import resnest50

from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss
import collections
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm
import time

In [40]:
# 看看label文件长啥样
labels_dataframe = pd.read_csv('./dataset/classify_leaves/train.csv')
labels_dataframe.head(5)
# labels_test=pd.read_csv('./dataset/classify_leaves/test.csv')
# labels_test

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [41]:
# 把label文件排个序
leaves_labels = sorted(list(set(labels_dataframe['label'])))
n_classes = len(leaves_labels)
n_classes

176

In [42]:
cls_to_num = dict(zip(leaves_labels, range(len(leaves_labels))))
num_to_cls = dict(zip(range(len(leaves_labels)), leaves_labels))

In [43]:
class TrainData(Dataset):
    def __init__(self, csv_path, img_path, transform,resize_h=224, resize_w=224):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.resize_h, self.resize_w = resize_h, resize_w
        self.img_path = img_path
        self.data_info = pd.read_csv(csv_path, header=None)
        self.data_len = len(self.data_info)
        self.train_img = np.asarray(self.data_info.iloc[1:self.data_len, 0])
        self.train_label = np.asarray(self.data_info.iloc[1:self.data_len, 1])
        self.transform=transform
        self.img_arr = self.train_img
        self.label_arr = self.train_label


    def __getitem__(self, index):
        img_name = self.img_arr[index]
        img_as_img = Image.open(os.path.join(self.img_path, img_name))
        # 如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        #         if img_as_img.mode != 'L':
        #             img_as_img = img_as_img.convert('L')
        # transform = transforms.Compose([
        #     # transforms.RandomResizedCrop((224,224),scale=(0.8,1.0),ratio=(0.8,1.2)),#随机裁剪
        #     # transforms.transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),#随机调整图片的亮度、对比度、饱和度、色调
        #     transforms.Resize((224,224)),#缩放图片
        #     transforms.ToTensor(), ]  # 将图片转换成Tensor
        # )
        img_as_tensor = self.transform(img_as_img)
        label = self.label_arr[index]
        num_label = cls_to_num[label]
        return img_as_tensor, num_label

    def __len__(self):
        return self.data_len-1

In [44]:
class TestData(Dataset):
    def __init__(self, csv_path, img_path, transform,resize_h=224, resize_w=224):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.resize_h, self.resize_w = resize_h, resize_w
        self.img_path = img_path
        self.transform=transform
        self.data_info = pd.read_csv(csv_path, header=None)
        self.data_len = len(self.data_info)
        self.test_img = np.asarray(self.data_info.iloc[1:self.data_len, 0])
        self.img_arr = self.test_img
       

    def __getitem__(self, index):
        img_name = self.img_arr[index]
        img_as_img = Image.open(os.path.join(self.img_path, img_name))
        # 如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        #         if img_as_img.mode != 'L':
        #             img_as_img = img_as_img.convert('L')
        # transform = transforms.Compose([
        #     # transforms.RandomResizedCrop((224,224),scale=(0.8,1.0),ratio=(0.8,1.2)),#随机裁剪
        #     # transforms.transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),#随机调整图片的亮度、对比度、饱和度、色调
        #     transforms.Resize((224,224)),#缩放图片
        #     transforms.ToTensor()]  # 将图片转换成Tensor
        # )
        img_as_tensor = self.transform(img_as_img)
        return img_as_tensor

    def __len__(self):
        return self.data_len-1

In [45]:
train_transform=transforms.Compose([
    transforms.RandomResizedCrop(224,scale=(0.08, 1.0),ratio=(3/4,4/3)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
                                   ])

test_transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [46]:
train_path = 'dataset/classify_leaves/train.csv'
test_path = 'dataset/classify_leaves/test.csv'
img_path = 'dataset/classify_leaves/'
train_set=TrainData(train_path,img_path,train_transform)
test_set=TestData(test_path,img_path,test_transform)
pd.read_csv(train_path)
# print(train_set.data_info)
# print(test_set.data_info)

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera
...,...,...
18348,images/18348.jpg,aesculus_glabra
18349,images/18349.jpg,liquidambar_styraciflua
18350,images/18350.jpg,cedrus_libani
18351,images/18351.jpg,prunus_pensylvanica


### ResNeSt模型

In [10]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False
# ResNest模型
def resnest_model(num_classes,feature_extracting=False):
    model=resnest50(pretrained=True)
    set_parameter_requires_grad(model,feature_extracting)
    num_ftrs=model.fc.in_features
    model.fc=nn.Sequential(nn.Linear(num_ftrs,num_classes))
    return model

In [11]:
device='cuda'

In [12]:
k_folds=5
num_epochs=30
learning_rate=1e-4
wd=1e-3
train_loss_function=CutMixCrossEntropyLoss(True)
test_loss_function=nn.CrossEntropyLoss()

results={}

torch.manual_seed(42)

kfold=KFold(n_splits=k_folds,shuffle=True)

#### 训练

In [ ]:
# Start print
print('--------------------------------------')

for fold,(train_ids,valid_ids) in enumerate(kfold.split(pd.read_csv(train_path))):
    print(f'{fold=}')
    print('*'*20)
    train_subsampler=torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler=torch.utils.data.SubsetRandomSampler(valid_ids)
    train_loader=torch.utils.data.DataLoader(
        CutMix(
            TrainData(train_path,img_path,transform=train_transform),
            num_class=176,beta=1.,prob=.5,num_mix=2),
        batch_size=32,
        sampler=train_subsampler,
        num_workers=4,
        pin_memory=False)
    valid_loader=torch.utils.data.DataLoader(
        TrainData(train_path,img_path,transform=test_transform),
        batch_size=32,
        sampler=valid_subsampler,
        num_workers=4,
        pin_memory=False)
    model=resnest_model(176).to(device)
    model.device=device
    optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay= wd)
    scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0, last_epoch=-1)
    # print(optimizer.param_groups)
    for epoch in range(num_epochs):
        time.sleep(0.5)
        model.train()
        print(f'Starting epoch {epoch+1}')
        train_loss = []
        train_acc = []
        for batch in tqdm(train_loader):
            time.sleep(0.05)
            imgs,labels=batch
            imgs,labels=imgs.to(device),labels.to(device)
            logits=model(imgs)
            loss = train_loss_function(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss.append(loss.item())         
        print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
        
        scheduler.step()
        train_avg_loss = np.sum(train_loss) / len(train_loss)
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_avg_loss:.5f}")
    print('Training process has finished. Saving trained model.')
    print('Starting validation')
    print('saving model with loss {:.3f}'.format(train_avg_loss))
    save_path = f'dataset/classify_leaves//model-fold-{fold}.pth'
    torch.save(model.state_dict(),save_path)
    
    model.eval()
    valid_loss = []
    valid_acc = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            imgs,labels=batch
            imgs,labels=imgs.to(device),labels.to(device)
            logits=model(imgs)
            loss = test_loss_function(logits, labels)
            valid_loss.append(loss.cpu().item())
            valid_acc.append((logits.argmax(dim=-1)==labels).float().mean().cpu())
        valid_avg_loss=np.sum(valid_loss)/len(valid_loss)
        valid_avg_acc=np.sum(valid_acc)/len(valid_acc)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_avg_loss:.5f}, acc = {valid_avg_acc:.5f}")
        print('Accuracy for fold %d: %.2f' % (fold, valid_avg_acc))
        results[fold]=valid_avg_acc
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation=0.
for key,value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation+=value
print(f'Average: {total_summation/len(results.items())} ')

--------------------------------------
fold=0
********************
Starting epoch 1


100%|██████████| 459/459 [01:57<00:00,  3.90it/s]


第1个epoch的学习率：0.000098
[ Train | 001/030 ] loss = 4.10735
Starting epoch 2


 99%|█████████▉| 454/459 [01:50<00:01,  4.28it/s]

In [21]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = test_transform),
                      batch_size=32, num_workers=4,pin_memory=False)

In [16]:
model = resnest_model(176)
model = model.to(device)
model.device=device
for test_fold in range(k_folds):
    model_path = f'dataset/classify_leaves/model-fold-{test_fold}.pth'
    saveFileName = f'dataset/classify_leaves/submission-fold-{test_fold}.csv'
    model.load_state_dict(torch.load(model_path))
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))
    preds=[]
    for batch in tqdm(testloader):
        imgs=batch
        imgs=imgs.to(device)
        with torch.no_grad():
            logits = tta_model(imgs)
        preds.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    test_df=pd.read_csv(test_path)
    test_df['label']=pd.Series(preds)
    submission=pd.concat([test_df['image'], test_df['label']], axis=1)
    submission.to_csv(saveFileName,index=False)
    print("ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

100%|██████████| 275/275 [01:03<00:00,  4.36it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:58<00:00,  4.68it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:58<00:00,  4.69it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:59<00:00,  4.66it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:58<00:00,  4.71it/s]

ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [17]:
df0 = pd.read_csv('dataset/classify_leaves/submission-fold-0.csv')
df1 = pd.read_csv('dataset/classify_leaves/submission-fold-1.csv')
df2 = pd.read_csv('dataset/classify_leaves/submission-fold-2.csv')
df3 = pd.read_csv('dataset/classify_leaves/submission-fold-3.csv')
df4 = pd.read_csv('dataset/classify_leaves/submission-fold-4.csv')

In [18]:
df_res=pd.concat([df0['label'],df1['label'],df2['label'],df3['label'],df4['label']],axis=1)
s_res=df_res.mode(axis=1)[0].apply(int)
submission=pd.concat([df0['image'], s_res], axis=1)
submission.columns=['image','label']
submission.to_csv('dataset/classify_leaves/submission-resnest.csv', index=False)
print('Voting results of resnest successfully saved!')

Voting results of resnest successfully saved!


### ResNeXt

In [22]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# resnext50_32x4d模型
def resnext_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.resnext50_32x4d(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [23]:
# Configuration options
device='cuda:0'
k_folds = 5
num_epochs = 30
lr = 1e-3
wd = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
test_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [ ]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids,valid_ids) in enumerate(kfold.split(pd.read_csv(train_path))):
    print(f'FOLD {fold}')
    print('*'*20)

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      CutMix(
                          TrainData(train_path, img_path, transform = train_transform), 
                          num_class=176, beta=1.0, prob=0.5, num_mix=2),
                      batch_size=128, sampler=train_subsampler, num_workers=4,pin_memory=False,drop_last=True)
    validloader = torch.utils.data.DataLoader(
                      TrainData(train_path, img_path, transform = test_transform),
                      batch_size=128, sampler=valid_subsampler, num_workers=4,pin_memory=False,drop_last=True)
    model = resnext_model(176)
    model = model.to(device)
    model.device = device
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr,weight_decay= wd)
    #   optimizer = SWA(our_optimizer, swa_start=5, swa_freq =5, swa_lr=0.05)
    scheduler = CosineAnnealingLR(optimizer,T_max=10)
    for epoch in range(0,num_epochs):
        time.sleep(.5)
        model.train()
        # Print epoch
        print(f'Starting epoch {epoch+1}')
        # These are used to record information in training
        train_losses = []
        train_accs = []
        # Iterate the training set by batches   
        for batch in tqdm(trainloader):
            time.sleep(0.005)
            # Move images and labels to GPU
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            # Forward the data
            logits = model(imgs)
            loss = train_loss_function(logits,labels)
            # Clear gradients in previous step
            optimizer.zero_grad()
            # Compute gradients for parameters
            loss.backward()
            optimizer.step()
            # Compute the accuracy for current batch.
            # acc = (logits.argmax(dim=-1) == labels).float().mean()
            # Record the loss and accuracy.
            train_losses.append(loss.item())
            time.sleep(0.005)
        print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
        scheduler.step()
        train_avg_loss = np.sum(train_losses) / len(train_losses)
        # train_acc = np.sum(train_accs) / len(train_accs)
        # Print the information.
        # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_avg_loss:.5f}")
    print('Training process has finished. Saving trained model.')
    print('Starting validation')

    # Saving the model
    print('saving model with loss {:.3f}'.format(train_avg_loss))
    save_path = f'dataset/classify_leaves/model2-fold-{fold}.pth'
    torch.save(model.state_dict(),save_path)
    # Start Validation
    model.eval()
    valid_losses = []
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(validloader):
            time.sleep(0.005)
            imgs, labels = batch
            # No gradient in validation
            logits = model(imgs.to(device))
            loss = test_loss_function(logits,labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean().cpu()
            # Record loss and accuracy
            valid_losses.append(loss.item())        
            valid_accs.append(acc)
        # The average loss and accuracy
        valid_avg_loss = np.sum(valid_losses)/len(valid_losses)
        valid_avg_acc = np.sum(valid_accs)/len(valid_accs)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_avg_loss:.5f}, acc = {valid_avg_acc:.5f}")
        print('Accuracy for fold %d: %d' % (fold, valid_avg_acc))
        print('--------------------------------------')
        results[fold] = valid_avg_acc
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
    print(f'Average: {total_summation/len(results.items())} ')

--------------------------------------
FOLD 0
********************
Starting epoch 1


100%|██████████| 114/114 [01:05<00:00,  1.74it/s]


第1个epoch的学习率：0.001000
[ Train | 001/030 ] loss = 4.10203
Starting epoch 2


100%|██████████| 114/114 [01:04<00:00,  1.75it/s]


第2个epoch的学习率：0.000976
[ Train | 002/030 ] loss = 3.35978
Starting epoch 3


...

100%|██████████| 114/114 [01:05<00:00,  1.75it/s]


第26个epoch的学习率：0.000500
[ Train | 026/030 ] loss = 1.82230
Starting epoch 27


100%|██████████| 114/114 [01:04<00:00,  1.77it/s]


第27个epoch的学习率：0.000345
[ Train | 027/030 ] loss = 1.74374
Starting epoch 28


 89%|████████▊ | 101/114 [00:57<00:07,  1.85it/s]

In [47]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = test_transform),
                      batch_size=128, num_workers=4,pin_memory=False)

In [25]:
## predict
model = resnext_model(176)

# create model and load weights from checkpoint
model = model.to(device)
model.device=device
# load the all folds
for test_fold in range(k_folds):
    model_path = f'dataset/classify_leaves/model2-fold-{test_fold}.pth'
    saveFileName = f'dataset/classify_leaves/submission2-fold-{test_fold}.csv'
    model.load_state_dict(torch.load(model_path))
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))

    # Initialize a list to store the predictions.
    preds = []
    # Iterate the testing set by batches.
    for batch in tqdm(testloader):
        imgs = batch
        with torch.no_grad():
            logits = tta_model(imgs.to(device))

        # Take the class with greatest logit as prediction and record it.
        preds.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    test_data = pd.read_csv(test_path)
    test_data['label'] = pd.Series(preds)
    submission = pd.concat([test_data['image'], test_data['label']], axis=1)
    submission.to_csv(saveFileName, index=False)
    print("ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

100%|██████████| 69/69 [00:47<00:00,  1.46it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [00:48<00:00,  1.43it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [00:47<00:00,  1.47it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [00:47<00:00,  1.45it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [00:47<00:00,  1.45it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [26]:
df0 = pd.read_csv('dataset/classify_leaves/submission2-fold-0.csv')
df1 = pd.read_csv('dataset/classify_leaves/submission2-fold-1.csv')
df2 = pd.read_csv('dataset/classify_leaves/submission2-fold-2.csv')
df3 = pd.read_csv('dataset/classify_leaves/submission2-fold-3.csv')
df4 = pd.read_csv('dataset/classify_leaves/submission2-fold-4.csv')
df_res=pd.concat([df0['label'],df1['label'],df2['label'],df3['label'],df4['label']],axis=1)
s_res=df_res.mode(axis=1)[0].apply(int)
submission=pd.concat([df0['image'], s_res], axis=1)
submission.columns=['image','label']
submission.to_csv('dataset/classify_leaves/submission2-resnext.csv', index=False)
print('Voting results of resnest successfully saved!')
#多众数投票到最小的那个

Voting results of resnest successfully saved!


In [27]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# densenet161模型
def dense_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.densenet161(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [37]:
# Configuration options
k_folds = 5
num_epochs = 30
learning_rate = 1e-4
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [ ]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids,valid_ids) in enumerate(kfold.split(pd.read_csv(train_path))):
    print(f'FOLD {fold}')
    print('*'*20)

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      CutMix(
                          TrainData(train_path, img_path, transform = train_transform), 
                          num_class=176, beta=1.0, prob=0.5, num_mix=2),
                      batch_size=32, sampler=train_subsampler, num_workers=4,pin_memory=False,drop_last=True)
    validloader = torch.utils.data.DataLoader(
                      TrainData(train_path, img_path, transform = test_transform),
                      batch_size=32, sampler=valid_subsampler, num_workers=4,pin_memory=False,drop_last=True)
    model = dense_model(176)
    model = model.to(device)
    model.device = device
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr,weight_decay= wd)
    #   optimizer = SWA(our_optimizer, swa_start=5, swa_freq =5, swa_lr=0.05)
    scheduler = CosineAnnealingLR(optimizer,T_max=10)
    for epoch in range(0,num_epochs):
        time.sleep(.5)
        model.train()
        # Print epoch
        print(f'Starting epoch {epoch+1}')
        # These are used to record information in training
        train_losses = []
        train_accs = []
        # Iterate the training set by batches   
        for batch in tqdm(trainloader):
            time.sleep(.005)
            # Move images and labels to GPU
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            # Forward the data
            logits = model(imgs)
            loss = train_loss_function(logits,labels)
            # Clear gradients in previous step
            optimizer.zero_grad()
            # Compute gradients for parameters
            loss.backward()
            optimizer.step()
            # Compute the accuracy for current batch.
            # acc = (logits.argmax(dim=-1) == labels).float().mean()
            # Record the loss and accuracy.
            train_losses.append(loss.item())
            time.sleep(.005)
        print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
        scheduler.step()
        train_avg_loss = np.sum(train_losses) / len(train_losses)
        # train_acc = np.sum(train_accs) / len(train_accs)
        # Print the information.
        # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_avg_loss:.5f}")
    print('Training process has finished. Saving trained model.')
    print('Starting validation')

    # Saving the model
    print('saving model with loss {:.3f}'.format(train_avg_loss))
    save_path = f'dataset/classify_leaves/model3-fold-{fold}.pth'
    torch.save(model.state_dict(),save_path)
    # Start Validation
    model.eval()
    valid_losses = []
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(validloader):
            time.sleep(.005)
            imgs, labels = batch
            # No gradient in validation
            logits = model(imgs.to(device))
            loss = valid_loss_function(logits,labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean().cpu()
            # Record loss and accuracy
            valid_losses.append(loss.item())        
            valid_accs.append(acc)
            time.sleep(.005)
        # The average loss and accuracy
        valid_avg_loss = np.sum(valid_losses)/len(valid_losses)
        valid_avg_acc = np.sum(valid_accs)/len(valid_accs)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_avg_loss:.5f}, acc = {valid_avg_acc:.5f}")
        print('Accuracy for fold %d: %d' % (fold, valid_avg_acc))
        print('--------------------------------------')
        results[fold] = valid_avg_acc
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
    print(f'Average: {total_summation/len(results.items())} ')

--------------------------------------
FOLD 0
********************
Starting epoch 1


100%|██████████| 458/458 [02:12<00:00,  3.46it/s]


第1个epoch的学习率：0.001000
[ Train | 001/030 ] loss = 5.12747
Starting epoch 2


100%|██████████| 458/458 [02:08<00:00,  3.56it/s]


第2个epoch的学习率：0.000976
[ Train | 002/030 ] loss = 4.92846
Starting epoch 3


...

100%|██████████| 114/114 [00:25<00:00,  4.45it/s]


[ Valid | 030/030 ] loss = 1.61269, acc = 0.85362
Accuracy for fold 0: 0
--------------------------------------
FOLD 1
********************
Starting epoch 1


100%|██████████| 458/458 [03:33<00:00,  2.14it/s]


第1个epoch的学习率：0.001000
[ Train | 001/030 ] loss = 4.92309
Starting epoch 2


 64%|██████▍   | 292/458 [01:59<00:46,  3.60it/s]

In [48]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = test_transform),
                      batch_size=128, num_workers=4,pin_memory=False)

In [49]:
model = dense_model(176)

# create model and load weights from checkpoint
model = model.to(device)
model.device=device
# load the all folds
for test_fold in range(k_folds):
    model_path = f'dataset/classify_leaves/model3-fold-{test_fold}.pth'
    saveFileName = f'dataset/classify_leaves/submission3-fold-{test_fold}.csv'
    model.load_state_dict(torch.load(model_path))
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))

    # Initialize a list to store the predictions.
    preds = []
    # Iterate the testing set by batches.
    for batch in tqdm(testloader):
        imgs = batch
        with torch.no_grad():
            logits = tta_model(imgs.to(device))

        # Take the class with greatest logit as prediction and record it.
        preds.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    test_data = pd.read_csv(test_path)
    test_data['label'] = pd.Series(preds)
    submission = pd.concat([test_data['image'], test_data['label']], axis=1)
    submission.to_csv(saveFileName, index=False)
    print("Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

100%|██████████| 69/69 [01:25<00:00,  1.24s/it]


Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:26<00:00,  1.25s/it]


Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:26<00:00,  1.25s/it]


Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:26<00:00,  1.26s/it]


Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:26<00:00,  1.25s/it]


Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [50]:
df0 = pd.read_csv('dataset/classify_leaves/submission3-fold-0.csv')
df1 = pd.read_csv('dataset/classify_leaves/submission3-fold-1.csv')
df2 = pd.read_csv('dataset/classify_leaves/submission3-fold-2.csv')
df3 = pd.read_csv('dataset/classify_leaves/submission3-fold-3.csv')
df4 = pd.read_csv('dataset/classify_leaves/submission3-fold-4.csv')
df_res=pd.concat([df0['label'],df1['label'],df2['label'],df3['label'],df4['label']],axis=1)
s_res=df_res.mode(axis=1)[0].apply(int)
submission=pd.concat([df0['image'], s_res], axis=1)
submission.columns=['image','label']
submission.to_csv('dataset/classify_leaves/submission3-densenet.csv', index=False)
print('Voting results of resnest successfully saved!')

Voting results of resnest successfully saved!


In [62]:
df_resnest = pd.read_csv('dataset/classify_leaves/submission-resnest.csv')
df_resnext = pd.read_csv('dataset/classify_leaves/submission2-resnext.csv')
df_densenet = pd.read_csv('dataset/classify_leaves/submission3-densenet.csv')
df_res=pd.concat([df_resnest['label'],df_resnext['label'],df_densenet['label']],axis=1)
np_res=np.array(df_res.values)
res=[]
for i in range(len(df_res)):
    tmp=np_res[i]
    # print(tmp)
    tmp_m=collections.Counter(tmp)
    li=sorted([*zip(tmp_m.keys(),tmp_m.values())],key=lambda x:-x[1])
    # print(li)
    if len(li)==1 or li[1][1]<li[0][1]:
        res.append(li[0][0])
    else:
        res.append(tmp[0])
final_res=pd.Series([num_to_cls[i] for i in res],name='label')
final_df=pd.concat([df_resnest['image'], final_res], axis=1)
final_df.to_csv('dataset/classify_leaves/final_submission.csv', index=False)

In [41]:
df0 = pd.read_csv('dataset/classify_leaves/submission-fold-0.csv')
df1 = pd.read_csv('dataset/classify_leaves/submission-fold-1.csv')

In [42]:
df0

,image,label
0,images/18353.jpg,22
1,images/18354.jpg,123
2,images/18355.jpg,150
3,images/18356.jpg,110
4,images/18357.jpg,121
...,...,...
8795,images/27148.jpg,112
8796,images/27149.jpg,162
8797,images/27150.jpg,112
8798,images/27151.jpg,78


In [17]:
a=a.to(device)

In [22]:
a.mean().cpu().item()

2.0